# About:

Prepare the data to populate the 'TexteFiscExactCode' column from the 'Textes' table.

In [1]:
# !pip install llama-index
# !pip install llama-index-readers-database
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-llms-ollama
# !pip install llama-index-postprocessor-cohere-rerank
# !pip install llama-index-postprocessor-flag-embedding-reranker
# !pip install FlagEmbedding
# !pip install openpyxl
# !pip install psycopg2
# !pip install pandas
# !pip install sqlalchemy
# !pip install tiktoken

In [ ]:
# !pip install llama-index-llms-openai

In [2]:
# !pip install pyvis

In [3]:
# !pip install llama-index-embeddings-ollama
# !pip install llama-index-postprocessor-colbert-rerank

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
COUNTRY_NAME = 'MLI Mali'
SPOKEN_LANGUAGE = "French"

In [6]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print('country name OK')

country name OK


## Get data from Postgres

In [7]:
from postgres_connection import get_postgress_data
from sql_files import sql_files
import pandas as pd

### This is not necessary if we extract the data using the llama_index db reader

In [8]:
df = get_postgress_data(sql_files['get_docs_per_country'].replace("%country_name%", COUNTRY_NAME))

/home/andrei/Ferdi_LLM/postgres_connection.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
df.shape

(218, 3)

In [10]:
df_droit = get_postgress_data(sql_files['get_droit_type_for_docs_per_country'].replace("%country_name%", COUNTRY_NAME))

In [11]:
df_droit.shape

(218, 2)

In [12]:
df = pd.merge(df,df_droit,how='left', on='title')

### This is mandatory

In [13]:
df_fisc_exacts = get_postgress_data(sql_files['get_textes_fiscaux_exacts'], db='Ferdi')

/home/andrei/Ferdi_LLM/postgres_connection.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [14]:
df_fisc_exacts

,TexteFiscExactCode,TexteFiscStandardCode,TexteFiscExactCodeCourt,TexteFiscExactComplet
0,_Gen_CGI_CGI,_Gen_CGI,_CGI,Code général des impôts
1,_Gen_CGI_CGImodif,_Gen_CGI,_CGImodif,Modification du Code général des impôts
2,_Gen_CGI_CGImodifModif,_Gen_CGI,_CGImodifModif,Modification de la Modification du Code généra...
3,_Gen_CGI_CGIamendé,_Gen_CGI,_CGIamendé,Code général des impôts amendé
4,_Gen_CGI_CGIappli,_Gen_CGI,_CGIappli,Application du Code général des impôts
...,...,...,...,...
65,_Pétrol_CP_CPappliModif,_Pétrol_CP,_CPappliModif,Modification de l'Application du Code pétrolier
66,_Pétrol_CP_CPconv,_Pétrol_CP,_CPconv,Convention pétrolière type
67,_Pétrol_CP_CPconvModif,_Pétrol_CP,_CPconvModif,Modification de la Convention pétrolière type
68,_Gen_CGI_CGI2,_Gen_CGI,_CGI,Code général des impôts2


In [15]:
df_fisc_standards = get_postgress_data(sql_files['get_textes_fiscaux_standards'], db='Ferdi')

/home/andrei/Ferdi_LLM/postgres_connection.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [16]:
df_fisc_standards.head()

,TexteFiscStandardCode,RégimeCode,TexteFiscStandardCodeCourt,TexteFiscStandardComplet
0,_Gen_CGI,_Gen,_CGI,Code général des impôts
1,_Gen_IT,_Gen,_IT,Impôt sur le revenu
2,_Gen_VAT,_Gen,_VAT,Taxe sur la valeur ajoutée
3,_Gen_NHI,_Gen,_NHI,Assurance maladie nationale
4,_Gen_LF,_Gen,_LF,Loi de finances


# RAG approach

## Get data from Postgres using llama-index db reader

In [17]:
from postgres_connection import psql_conn_config
from llama_index.readers.database import DatabaseReader
from sql_files import sql_files

In [18]:
db = DatabaseReader(
    scheme="postgresql",  # Database Scheme
    host=psql_conn_config.get("HOSTNAME"),  # Database Host
    port="5432",  # Database Port
    user=psql_conn_config.get("USERNAME"),  # Database User
    password=psql_conn_config.get("PASSWORD"),  # Database Password
    dbname=psql_conn_config.get("DATABASE"),  # Database Name
)

### Load the data as llama_index documents

In [19]:
from llama_index.core import Document, VectorStoreIndex

# documents = db.load_data(query=sql_files['get_docs_MALI'])

# texts = df[~df['title'].str.contains("JO")][df['content'].str.len()<2000000].to_dict(orient='index')
texts = df[~df['title'].str.contains("JO")].to_dict(orient='index')
# texts = dict(list(texts.items())[-30:])
documents = [Document(text=txt['content'], metadata={'title': txt['title'], 'law type': txt['tag']}) for txt in texts.values()]

## RAG

### Embeddings model

#### Hugging Face embedding

In [20]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from sentence_transformers import SentenceTransformer

# embed_model = HuggingFaceEmbedding(model_name="dunzhang/stella_en_1.5B_v5", trust_remote_code=True) #Alibaba-NLP/gte-Qwen2-1.5B-instruct
# embed_model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)

You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/home/andrei/anaconda3/envs/Ferdi_env_2/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Vector DataBase

In [21]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

import os
from llama_index.core import VectorStoreIndex, load_index_from_storage
from llama_index.core.storage import StorageContext
from llama_index.core import Settings

# Settings.embed_model = embed_model # we specify the embedding model to be used

from llama_index.core.node_parser import TokenTextSplitter


In [111]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
embed_model = OpenAIEmbedding(model="text-embedding-3-small") 
llm = OpenAI(model="gpt-4o", temperature=0)

os.environ["OPENAI_API_KEY"] = "sk-5adfhvA2R083PAZKReymxs2sZQu-DkjA4i0cD_zKm7T3BlbkFJRQ4j--mKB6fsRJjagdOMgo6H37A4LnfAddP2VyNxwA"

In [112]:
from llama_index.core import Settings

Settings.embed_model = embed_model

In [113]:
from llama_index.core.text_splitter import SentenceSplitter

In [114]:
chunk_size = int(1024)
overlap = 100 if chunk_size > 1024 else int(chunk_size/10)

transformations_example = [
    TokenTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size/10),
        separator=" ",
    ),
    embed_model,
]

transformations_sentence = [SentenceSplitter(chunk_size=chunk_size, chunk_overlap=overlap)]

storage_name = "storage_MLI Mali_complete_1024_openai_small"

if not os.path.exists(storage_name):
    index = VectorStoreIndex.from_documents(documents, show_progress=True, transformations=transformations_example)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist(f"./{storage_name}")
else:
    print('loading from local')
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir=storage_name)
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

vector_index = index

loading from local


In [ ]:
chunk_size = int(4096)

transformations_example = [
    TokenTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size/10),
        separator=" ",
    ),
    embed_model,
]

if not os.path.exists("storage_MLI Mali_w_metadata_4096_full"):
    index = VectorStoreIndex.from_documents(documents, transformations=transformations_example)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage_MLI Mali_w_metadata_4096_full")
else:
    print('loading from local')
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage_MLI Mali_w_metadata_4096_full")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

In [115]:
doc_titles = pd.Series(list(set([v.get('title') for k,v in index.vector_store.to_dict()['metadata_dict'].items()])))

#### Query pipeline

In [116]:
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import QueryPipeline

from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer

from llama_index.core.response_synthesizers import TreeSummarize

In [117]:
# setting up the llm
# llm = Ollama(
#     model="llama3.1", 
#     temperature=0.01, 
#     request_timeout=180.0,
#     # context_window=chunk_size,
#     context_window=2**14) 

Settings.llm = llm

In [118]:
definitions = df_fisc_standards.set_index(['TexteFiscStandardCode'])['TexteFiscStandardComplet'].to_dict()
definitions.keys()

dict_keys(['_Gen_CGI', '_Gen_IT', '_Gen_VAT', '_Gen_NHI', '_Gen_LF', '_Invest_CI', '_Min_CM', '_Pétrol_CP', '_Gaz_CG', '_Forest_CF'])

In [119]:
# retriever = index.as_retriever(similarity_top_k=5)
summarizer = TreeSummarize(llm=llm)

In [120]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.query_pipeline import InputComponent


In [121]:
vector_index = index

In [122]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)

from typing import List

In [123]:
# doc_name = 'Ordonnance n°2020-013 (21.12.2020) Loi de finances 2021 (Ministère des Finances)'
# doc_name = "Loi n°2001-075 (18.07.2001) Code des douanes 2001 (eRegulations Mali)"
# doc_name ="img_Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)" # this document contains  MODIFICATION DU CODE GENERAL DES IMPOTS and Les articles 73 (nouveau) et 74 A du Code Général des Impôts sont modifiés ainsi qu’il suit

# doc_name = "Décret n°2004-357 (08.09.2004) Application Code pétrolier 2004 (Droit-Afrique)"
# doc_name = "img_Décret n°2019-006 (10.01.2019) ITIE 2019 (ITIE Mali)"
# doc_name = "img_Loi n°2023-041 (29.08.2023) Contenu local minier 2023 (LinkedIn)"
# doc_name = 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé'
# doc_name = 'Loi n°2019-070 (24.12.2019) Loi de finances 2020 (Ministère des Finances)'
doc_name = 'Décret n°1997-182 (02.06.1997) Application Jeux de hasard 1997 (SGG)'
doc_name = 'img_Loi n°2016-056 (21.12.2016) Loi de finances 2017 (Ministère des Finances)'

#### Trial 1

In [124]:
from llama_index.core.postprocessor import SimilarityPostprocessor

processor = SimilarityPostprocessor(similarity_cutoff=0.5)
# filtered_nodes = processor.postprocess_nodes(nodes)

In [125]:
from llama_index.core.postprocessor import LLMRerank

from llama_index.postprocessor.colbert_rerank import ColbertRerank

In [126]:
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.query_engine import RetrieverQueryEngine

vector_index = index

from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

def create_vector_engine(doc_name):
    filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key="title", value=doc_name
            ),
        ]
    )

    reranker = LLMRerank(
                llm=llm,
                choice_batch_size=10,
                top_n=5,
            )

    colbert_reranker = ColbertRerank(
        top_n=5,
        model="colbert-ir/colbertv2.0",
        tokenizer="colbert-ir/colbertv2.0",
        keep_retrieval_score=True,
    )

    vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=10, filters=filters) #, alpha=0.5) #, embed_model=embed_model)
    response_synthesizer = get_response_synthesizer(llm=llm, response_mode=ResponseMode.COMPACT)

    vector_query_engine = RetrieverQueryEngine(
        retriever=vector_retriever,
        response_synthesizer=response_synthesizer,
        # node_postprocessors=[colbert_reranker],
        # node_postprocessors=[processor]
    )

    return vector_query_engine

In [127]:
df_fisc_exacts.set_index('TexteFiscExactCode')['TexteFiscExactComplet'].to_dict()

{'_Gen_CGI_CGI': 'Code général des impôts',
 '_Gen_CGI_CGImodif': 'Modification du Code général des impôts',
 '_Gen_CGI_CGImodifModif': 'Modification de la Modification du Code général des impôts',
 '_Gen_CGI_CGIamendé': 'Code général des impôts amendé',
 '_Gen_CGI_CGIappli': 'Application du Code général des impôts',
 '_Gen_CGI_CGIappliModif': "Modification de l'Application du Code général des impôts",
 '_Gen_CGI_CGIsuppl': 'Supplément au Code général des impôts',
 '_Gen_CGI_CGIsupplAppli': 'Application du Supplément au Code général des impôts',
 '_Gen_CGI_CGIone': 'Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneModif': 'Modification du Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneAmendé': 'Code des impôts directs et indirects amendé',
 '_Gen_CGI_CGIbis': "Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobiliers",
 '_Gen_CGI_CGIbisModif': "Modification du Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobil

In [128]:
def get_large_categ(doc_name, definitions=definitions):
    vector_query_engine = create_vector_engine(doc_name)

    response = vector_query_engine.query(
        f"""
        Consider the contents of the document with this title '{doc_name}'. Then, these definitions: {str(definitions)} 
        In which of these categories : {', '.join(list(definitions.keys()))}, is it part of ? Answer with a valid category, keep the category only. Do not write an introduction, summary, rewrite or anything after the category.
        """
        # Answer with a valid category, keep the category only.
        # f"""
        # Considérez le contenu du document portant ce nom '{doc_name}'. Ensuite, ces définitions : {str(definitions)}
        # Dans laquelle de ces catégories : {', '.join(list(definitions.keys()))}, cela en fait-il partie ? Répondez avec une catégorie valide, gardez uniquement la catégorie.
        # """
    )

    # print(response)
    print(doc_name, len(response.source_nodes))

    large_cat = response.response

    if large_cat[-1] == "_":
        large_cat = large_cat[-1]

    definitions2 = df_fisc_exacts[df_fisc_exacts['TexteFiscStandardCode']==large_cat].set_index(['TexteFiscExactCode'])['TexteFiscExactComplet'].to_dict()
    print(definitions2)

    response2 = vector_query_engine.query(
        f"""
        The following represents a list of definitions, where we define categories, followed by '=' and then the definitions: {", ".join([f"{k}={v}" for k,v in definitions2.items()])}
        Consider the contents of the document with this name '{doc_name}'.
        In which category, is it part of ? Answer with a valid category, keep the category only. Do not write an introduction, summary, rewrite or anything after the category.
        """

        # f"""
        # Consider the contents of the document with this name '{doc_name}'.
        # In which of these categories : {', '.join(list(definitions2.keys()))}, is it part of ? Answer with a valid category, keep the category only. Do not write an introduction, summary, rewrite or anything after the category.
        # """
        # 
        #  Please reply in French
        # f"""
        # Ce qui suit représente une liste de définitions, où nous définissons les catégories, suivies de '=' puis des définitions : {", ".join([f"{k}={v}" for k,v in definitions2.items()])}
        # Considérez le contenu du document portant ce nom '{doc_name}'.
        # Dans laquelle de ces catégories : {', '.join(list(definitions2.keys()))}, cela en fait-il partie ? Répondez avec une catégorie valide, gardez uniquement la catégorie.
        # """
    )

    return response2.response

In [129]:
response = get_large_categ('Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé') # doc_titles[6]

print(response)

Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé 10
{'_Gen_CGI_CGI': 'Code général des impôts', '_Gen_CGI_CGImodif': 'Modification du Code général des impôts', '_Gen_CGI_CGImodifModif': 'Modification de la Modification du Code général des impôts', '_Gen_CGI_CGIamendé': 'Code général des impôts amendé', '_Gen_CGI_CGIappli': 'Application du Code général des impôts', '_Gen_CGI_CGIappliModif': "Modification de l'Application du Code général des impôts", '_Gen_CGI_CGIsuppl': 'Supplément au Code général des impôts', '_Gen_CGI_CGIsupplAppli': 'Application du Supplément au Code général des impôts', '_Gen_CGI_CGIone': 'Code des impôts directs et indirects', '_Gen_CGI_CGIoneModif': 'Modification du Code des impôts directs et indirects', '_Gen_CGI_CGIoneAmendé': 'Code des impôts directs et indirects amendé', '_Gen_CGI_CGIbis': "Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobiliers", '_Gen_CGI_CGIbisModif': "Modification du Code de l'enreg

In [130]:
large_categs = list()

for doc in doc_titles:
    try:
        categ = get_large_categ(doc)
    except Exception as e:
        print(e)

        categ = "non_determined"

    large_categs.append(categ)

    

img_Ordonnance n°1991-065 (19.09.1991) Code minier 1991 (Ministère des Mines) 10
{'_Min_CM_CM': 'Code minier', '_Min_CM_CMmodif': 'Modification du Code minier', '_Min_CM_CMamendé': 'Code minier amendé', '_Min_CM_CMratif': 'Ratification du Code minier', '_Min_CM_CMappli': 'Application du Code minier', '_Min_CM_CMappliModif': "Modification de l'Application du Code minier", '_Min_CM_CMconv': 'Convention minière type', '_Min_CM_CMconvModif': 'Modification de la Convention minière type', '_Min_CM_CMfisc': 'Fiscalité minière', '_Min_CM_CMfiscModif': 'Modification de la Fiscalité minière', '_Min_CM_CMsuppl': 'Supplément au Code minier', '_Min_CM_CMitie': 'Initiative pour la Transparence dans les Industries Extractives'}
img_Décret n°2018-685 (31.08.2018) ITIE 2018 (ITIE Mali) 3
{'_Min_CM_CM': 'Code minier', '_Min_CM_CMmodif': 'Modification du Code minier', '_Min_CM_CMamendé': 'Code minier amendé', '_Min_CM_CMratif': 'Ratification du Code minier', '_Min_CM_CMappli': 'Application du Code minier

In [131]:
dict(zip(doc_titles, large_categs))

{'img_Ordonnance n°1991-065 (19.09.1991) Code minier 1991 (Ministère des Mines)': '_Min_CM_CM=Code minier',
 'img_Décret n°2018-685 (31.08.2018) ITIE 2018 (ITIE Mali)': '_Min_CM_CMitie',
 'img_Décret n°1999-255 (15.09.1999) Application Code minier 1999 (Ministère des Mines)': '_Min_CM_CMappli',
 'Décret n°2004-357 (08.09.2004) Application Code pétrolier 2004 (Ministère des Mines)': '_Pétrol_CP_CPappli',
 'img_Décret n°2016-520 (22.07.2016) ITIE 2016 (ITIE Mali)': 'droit minier',
 'img_Loi n°2004-037 (02.08.2004) Code pétrolier 2004 (Droit-Afrique)': '_Pétrol_CP_CP',
 'img_Loi n°2010-060 (30.12.2010) Loi de finances 2011 (Droit-Afrique)': '_Gen_LF_LF',
 'Loi n°2001-075 (18.07.2001) Code des douanes 2001 (eRegulations Mali)': 'The document is not part of any category listed.',
 'Décret n°2004-357 (08.09.2004) Application Code pétrolier 2004 (Droit-Afrique)': '_Pétrol_CP_CPappli',
 'Loi n°2015-054 (22.12.2015) Annexe fiscale Loi de finances 2016 (SEAG Conseil)': '_Gen_CGI_CGIprocedModif',

In [ ]:
set(large_categs)

{'_Gen_CGI',
 '_Gen_IT',
 '_Gen_LF',
 '_Gen_LF_',
 '_Gen_VAT',
 '_Min_CM',
 '_Pétrol_CP'}

In [ ]:
df_fisc_exacts['TexteFiscStandardCode'].unique()

array(['_Gen_CGI', '_Gen_IT', '_Gen_VAT', '_Gen_LF', '_Min_CM',
       '_Pétrol_CP'], dtype=object)

In [ ]:
print(f"{len(response.source_nodes)}")

for _node in response.source_nodes:
    print(f"{_node.score}\n{_node.text[:100]}")

5
0.7510572671890259
Mlle DIARRA  
PRESIDENCE DE LA REPUBLIQUE  
SECRETARIAT GENERAL  

REPUBLIQUE DU MALI  
Un Peuple - 
0.7502212524414062
le Contenu local.

CHAPITRE IV : DE LA MISE EN ŒUVRE ET DU SUIVI DU CONTENU LOCAL

Article 10 : Le C
0.7459142208099365
inclusion sur la plateforme d’appel à concurrence et l’interdiction de conclure des marchés liés aux
0.7431255578994751
les services de consultants et d’assistances techniques ;</li>
<li>h) la gestion des projets pour la
0.7271713614463806
<td>100</td>
  </tr>
  <tr>
    <td>16</td>
    <td>Forage hydraulique</td>
    <td>80</td>
    <td>


In [ ]:
definitions2 = df_fisc_exacts[df_fisc_exacts['TexteFiscStandardCode']==str('_Gen_CGI')].set_index(['TexteFiscExactCode'])['TexteFiscExactComplet'].to_dict()

definitions2

{'_Gen_CGI_CGI': 'Code général des impôts',
 '_Gen_CGI_CGImodif': 'Modification du Code général des impôts',
 '_Gen_CGI_CGImodifModif': 'Modification de la Modification du Code général des impôts',
 '_Gen_CGI_CGIamendé': 'Code général des impôts amendé',
 '_Gen_CGI_CGIappli': 'Application du Code général des impôts',
 '_Gen_CGI_CGIappliModif': "Modification de l'Application du Code général des impôts",
 '_Gen_CGI_CGIsuppl': 'Supplément au Code général des impôts',
 '_Gen_CGI_CGIsupplAppli': 'Application du Supplément au Code général des impôts',
 '_Gen_CGI_CGIone': 'Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneModif': 'Modification du Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneAmendé': 'Code des impôts directs et indirects amendé',
 '_Gen_CGI_CGIbis': "Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobiliers",
 '_Gen_CGI_CGIbisModif': "Modification du Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobil

In [ ]:
doc_name = 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP)'

vector_query_engine = create_vector_engine(doc_name)

response2 = vector_query_engine.query(
    f"""
    Consider the contents of the document with this title '{doc_name}'. Then, these definitions: {str(definitions2)} 
    In which category, is it part of ? Answer with a valid category, keep the category only. Do not write an introduction, summary, rewrite or anything after the category.
    """

    # The following represents a list of definitions, where we define categories, followed by '=' and then the definitions: {", ".join([f"{k}={v}" for k,v in definitions2.items()])}
    # Consider the contents of the document with this name '{doc_name}'.
    # In which category, is it part of ? Answer with a valid category, keep the category only. Do not write an introduction, summary, rewrite or anything after the category.

    #  Please reply in French
    # f"""
    # Ce qui suit représente une liste de définitions, où nous définissons les catégories, suivies de '=' puis des définitions : {", ".join([f"{k}={v}" for k,v in definitions2.items()])}
    # Considérez le contenu du document portant ce nom '{doc_name}'.
    # Dans laquelle de ces catégories : {', '.join(list(definitions2.keys()))}, cela en fait-il partie ? Répondez avec une catégorie valide, gardez uniquement la catégorie.
    # """
)

print(response2)

Token indices sequence length is longer than the specified maximum sequence length for this model (897 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (897) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
print(response2.source_nodes[1].text)

visibles pour les employés.
Art.27.- La demande d’autorisation de transfert Les titulaires de titres pétroliers et leurs sous-
suite à une cession ou d’amodiation doit être adres- traitants sont également tenus :
sée au Ministre dans les mêmes formes que celles • a) d’assurer aux travailleurs sur le site des
définies à l’Article 12 du présent décret. conditions d’hygiène et de salubrité conformes
à la législation en vigueur ;
Art.28.- Le transfert suite à une cession est accordé • b) de respecter la législation et les règlements
par décret pris en Conseil des Ministres, Il prend sanitaires ;
effet à compter de la date de signature du décret
• c) de respecter les conditions générales du tra-
d’autorisation de transfert.
vail relatives à la prévention et à la protection
contre les accidents de travail et les maladies
Art.29.- L’amodiation transfère à l’amodiataire
professionnelles ;
tous les droits et obligations de l’autorisation
• d) de contribuer à partir de la date de la pre-
d’explo

# Generate definitions from existing documents

In [ ]:
laws_dict = {
    "img_Décret n°2016-520 (22.07.2016) ITIE 2016 (ITIE Mali)": ["Initiative pour la Transparence dans les Insdustries Extractives", "_Min_CM_CMitie"],
    "img_Ordonnance n°1999-032 (19.08.1999) Code minier 1999 (Ministère des Mines)": ["Code minier", "_Min_CM_CM"],
    "img_Décret n°2012-311 (21.06.2012) Application Code minier 2012 (Ministère des Mines)": ["Application du Code minier", "_Min_CM_CMappli"],
    "Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)": ["Modification de la Convention minière type", "_Min_CM_CMconvModif"],
    "img_Loi n°2023-040 (29.08.2023) Code minier 2023 (LinkedIn)": ["Code minier", "_Min_CM_CM"],
    "img_Décret n°1999-256 (15.09.1999) Convention minière type 1999 (Ministère des Mines)": ["Convention minière type", "_Min_CM_CMconv"],
    "img_Loi n°2012-015 (27.02.2012) Code minier 2012 (Ministère des Mines)": ["Code minier", "_Min_CM_CM"],
    "img_Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (Ministère des Mines)": ["Modification de la Convention minière type", "_Min_CM_CMconvModif"],
    "Décret n°1999-256 (15.09.1999) Convention minière type 1999 (SGG)": ["Convention minière type", "_Min_CM_CMconv"],
    "Décret n°1999-255 (15.09.1999) Application Code minier 1999 (SGG)": ["Application du Code minier", "_Min_CM_CMappli"],
    "img_Loi n°2023-041 (29.08.2023) Contenu local minier 2023 (LinkedIn)": ["Supplément au Code minier", "_Min_CM_CMsuppl"],
    "img_Décret n°2007-180 (06.06.2007) ITIE 2007 (ITIE Mali)": ["Initiative pour la Transparence dans les Insdustries Extractives", "_Min_CM_CMitie"],
    "img_Décret n°1999-255 (15.09.1999) Application Code minier 1999 (Ministère des Mines)": ["Application du Code minier", "_Min_CM_CMappli"],
    "img_Décret n°2018-685 (31.08.2018) ITIE 2018 (ITIE Mali)": ["Initiative pour la Transparence dans les Insdustries Extractives", "_Min_CM_CMitie"],
    "img_Ordonnance n°2000-013 (10.02.2000) Modification Code minier 2000 (Ministère des Mines)": ["Modification du Code minier", "_Min_CM_CMmodif"],
    "img_Ordonnance n°1991-065 (19.09.1991) Code minier 1991 (Ministère des Mines)": ["Code minier", "_Min_CM_CM"],
    "Ordonnance n°2000-013 (10.02.2000) Modification Code minier 2000 (SGG)": ["Modification du Code minier", "_Min_CM_CMmodif"],
    "img_Décret n°2019-006 (10.01.2019) ITIE 2019 (ITIE Mali)": ["Initiative pour la Transparence dans les Insdustries Extractives", "_Min_CM_CMitie"],
    "img_Décret n°2020-288 (08.12.2020) Convention minière type 2020 (ITIE Mali)": ["Convention minière type", "_Min_CM_CMconv"],
    "img_Loi n°2008-027 (23.07.2008) Modification Code pétrolier 2008 (Ministère des Mines)": ["Modification du Code pétrolier", "_Pétrol_CP_CPmodif"],
    "Décret n°2004-357 (08.09.2004) Application Code pétrolier 2004 (Ministère des Mines)": ["Application du Code pétrolier", "_Pétrol_CP_CPappli"],
    "img_Loi n°2015-035 (16.07.2015) Code pétrolier 2015 (Ministère des Mines)": ["Code pétrolier", "_Pétrol_CP_CP"],
    "Loi n°2015-035 (16.07.2015) Code pétrolier 2015 (Droit-Afrique)": ["Code pétrolier", "_Pétrol_CP_CP"],
    "Décret n°2004-357 (08.09.2004) Application Code pétrolier 2004 (Droit-Afrique)": ["Application du Code pétrolier", "_Pétrol_CP_CPappli"],
    "Contrat de partage de production type (Ministère des Mines)": ["Convention pétrolière type", "_Pétrol_CP_CPconv"],
    "Décret n°2016-272 (29.04.2016) Application Code pétrolier 2016 (ITIE Mali)": ["Application du Code pétrolier", "_Pétrol_CP_CPappli"],
    "img_Loi n°2004-037 (02.08.2004) Code pétrolier 2004 (Droit-Afrique)": ["Code pétrolier", "_Pétrol_CP_CP"],
    "img_Loi n°2017-073 (26.12.2017) Loi de finances 2018 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "img_Loi n°2008-021 (22.07.2008) Modification LPF 2008 (Droit-Afrique)": ["Modification du Livre de procédures fiscales", "_Gen_CGI_CGIprocedModif"],
    "img_Loi n°2012-063 (26.12.2012) Annexe fiscale Loi de finances 2013 (Droit-Afrique)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°2016-056 (21.12.2016) Annexe fiscale Loi de finances 2017 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°1996-052 (16.10.1996) Taxe touristique 1996 (SGG)": ["Supplément au Code général des impôts", "_Gen_CGI_CGIsuppl"],
    "Loi n°1996-021 (21.02.1966) Jeux de hasard 1996 (SGG)": ["Supplément au Code général des impôts", "_Gen_CGI_CGIsuppl"],
    "Loi n°2001-064 (09.07.2001) Modification CGI 2001 (SGG)": ["Modification du Code général des impôts", "_Gen_CGI_CGImodif"],
    "Décret n°1996-179 (19.06.1996) Application Office malien de l_habitat 1996 (SGG)": ["Application du Fonds logement", "_Gen_CGI_FondsLogementAppli"],
    "img_Loi n°1996-030 (12.06.1996) Office malien de l_habitat 1996 (Faolex)": ["Fonds logement", "_Gen_CGI_FondsLogement"],
    "img_Loi n°2006-068 (29.12.2006) Livre de procédures fiscales 2006": ["Livre de procédures fiscales", "_Gen_CGI_CGIproced"],
    "Loi n°2006-068 (29.12.2006) Livre de procédures fiscales 2013 (Ekladata)": ["Livre de procédures fiscales amendé", "_Gen_CGI_CGIprocedAmendé"],
    "img_Loi n°2008-009 (28.02.2008) Modification CGI 2008 (Droit-Afrique)": ["Modification du Code général des impôts", "_Gen_CGI_CGImodif"],
    "Ordonnance n°2020-001 (04.09.2020) Loi de finances rectificative 2020 (Ministère des Finances)": ["Loi de finances rectificative", "_Gen_LF_LFR"],
    "Ordonnance n°1970-006 (27.02.1970) Code général des impôts 1999 (CIPB)": ["Code général des impôts amendé", "_Gen_CGI_CGIamendé"],
    "img_Loi n°2006-067 (29.12.2006) Code général des impôts 2016 (DGI)": ["Code général des impôts amendé", "_Gen_CGI_CGIamendé"],
    "Décret n°1997-182 (02.06.1997) Application Jeux de hasard 1997 (SGG)": ["Application du Supplément au Code général des impôts", "_Gen_CGI_CGIsupplAppli"],
    "img_Loi n°2017-022 (12.06.2021) Cadre général des exonérations fiscales et douanières 2017 (DGI)": ["Supplément au Code général des impôts", "_Gen_CGI_CGIsuppl"],
    "img_Loi n°2008-020 (22.07.2008) Modification CGI 2008 (Droit-Afrique)": ["Modification du Code général des impôts", "_Gen_CGI_CGImodif"],
    "img_Loi n°2016-056 (21.12.2016) Loi de finances 2017 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°2015-054 (22.12.2015) Annexe fiscale Loi de finances 2016 (SEAG Conseil)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°2011-078 (23.12.2011) Annexe fiscale Loi de finances 2012 (SEAG Conseil)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°2012-063 (26.12.2012) Annexe fiscale Loi de finances 2013 (DGI)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°1997-014 (07.03.1997) Modification CGI 1997 (SGG)": ["Modification du Code général des impôts", "_Gen_CGI_CGImodif"],
    "img_Loi n°2010-060 (30.12.2010) Loi de finances 2011 (Droit-Afrique)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP)": ["Code général des impôts", "_Gen_CGI_CGI"],
    "Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé": ["Code général des impôts", "_Gen_CGI_CGI"],
    "Loi n°2019-070 (24.12.2019) Loi de finances 2020 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°1994-034 (09.06.1994) Modification CGI 1994 (SGG)": ["Modification du Code général des impôts", "_Gen_CGI_CGImodif"],
    "img_Décret n°2018-595 (24.07.2018) Application Cadre général des exonérations fiscales et douanières 2018 (DGI)": ["Application du Supplément au Code général des impôts", "_Gen_CGI_CGIsupplAppli"],
    "Loi n°2021-071 (23.12.2021) Loi de finances 2022 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°1997-024 (23.01.1997) Modification CGI 1997 (SGG)": ["Modification du Code général des impôts", "_Gen_CGI_CGImodif"],
    "Loi n°1996-030 (12.06.1996) Office malien de l_habitat 1996 (SGG)": ["Fonds logement", "_Gen_CGI_FondsLogement"],
    "Ordonnance n°2013-021 (03.12.2013) Annexe fiscale Loi de finances 2014 (Droit-Afrique)": ["Loi de finances", "_Gen_LF_LF"],
    "img_Loi n°2015-054 (22.12.2015) Loi de finances 2016 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "img_Loi n°2017-073 (26.12.2017) Annexe fiscale Loi de finances 2018 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "Décret n°2005-036 (27.01.2005) Impôt spécial sur certains produits 2005 (Droit-Afrique)": ["Impôt spécial sur certains produits", "_Gen_CGI_ISCP"],
    "img_Loi n°2011-078 (23.12.2011) Annexe fiscale Loi de finances 2012 (Investir-Afrique)": ["Loi de finances", "_Gen_LF_LF"],
    "img_Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)": ["Loi de finances", "_Gen_LF_LF"],
    "Ordonnance n°2020-013 (21.12.2020) Loi de finances 2021 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
    "Loi n°2018-072 (21.12.2018) Loi de finances 2019 (Ministère des Finances)": ["Loi de finances", "_Gen_LF_LF"],
}


In [ ]:
def generate_definitions(doc_name, definition):
        vector_query_engine = create_vector_engine(doc_name)

        x = vector_query_engine.query(f"""
                Use the contents and title of the document with this title '{doc_name}', to create a definition that will help identify similar documents,
                around this phrase: '{definition}', with the emphasis on the type of law, type of document and the functionality of the document.
                Skip the document name, country and the year of the document from the answer. Keep the summary only, without any rewrites, notes etc.""")

        return x.response


In [ ]:
generated_definitions = {k: [generate_definitions(k, v[0]), v[0], v[1]] for (k,v) in laws_dict.items()}

In [ ]:
generated_definitions

{'img_Décret n°2016-520 (22.07.2016) ITIE 2016 (ITIE Mali)': "This is a decree that establishes the institutional framework for the Initiative for Transparency in Extractive Industries (ITIE), a law related to mining rights. The document outlines the structure and functions of ITIE's committees and secretariat, ensuring transparency and accountability in extractive industries.",
 'img_Ordonnance n°1999-032 (19.08.1999) Code minier 1999 (Ministère des Mines)': '**Rewrite**\n\nThe Code Minier is a type of droit minier document that outlines regulations and procedures for mining activities, providing rules for exploration, exploitation, environmental protection, and guidelines for permits and licenses.',
 'img_Décret n°2012-311 (21.06.2012) Application Code minier 2012 (Ministère des Mines)': '**Rewrite**\n\nThis document outlines the conditions and modalities for applying the Code minier, detailing procedures for obtaining permits, authorizations, and concessions for mining activities, a

In [ ]:
definitions = {
    "_Gen_LF_LOLF": "An organic law relating to finance laws (LOLF) is a legislative text that complements and clarifies the constitutional rules that govern the development, execution, and control of finance laws. It is thus a special law that explains how finance laws must be drafted and implemented. In French-speaking countries, the name of an organic law relating to finance laws generally includes the words 'portant loi organique relative aux lois de finances' or simply 'relative aux lois de finances'.",
    "_Gen_LF_LOLFmodif": "A modification of an organic law relating to finance laws is a legislative text that amends the constitutional rules governing the development, execution, and control of finance laws. It is thus a law that modifies the organic law relating to finance laws. In French-speaking countries, the name of a modification of an organic law relating to finance laws generally includes the words 'portant modification de la loi organique relative aux lois de finances' or 'modifiant la loi organique relative aux lois de finances'.",
    "_Gen_LF_LF": "This document is a fiscal law that outlines the financial regulations for the upcoming year, serving as a comprehensive guide for budgeting and expenditure management within the government. The emphasis on fiscal responsibility and transparency suggests a focus on prudent financial planning and accountability. It details various sections, programs, and articles, providing specific allocations for personnel, biens et services, transfers, and investments, ensuring effective allocation of resources and maintaining a stable economic environment.",
    "_Gen_LF_LFmodif": "A modification of a finance law is a legislative text (law or ordinance) aimed at amending a previously adopted finance law, without constituting a true rectifying finance law. In French-speaking countries, the name of a modification of a finance law generally includes the words 'portant modification de la loi de finances' or 'portant modification du budget général de l'État'. In English-speaking countries, a modification of a finance law is called a 'Finance (Amendment) Act'.",
    "_Gen_LF_LFamende": "An amended finance law is a modified version of a finance law up to a certain date. This amended version is based on the initial finance law and subsequently includes all subsequent modifications.",
    "_Gen_LF_LFratif": "The ratification of a finance law is a law passed to ratify an initial finance law adopted in the form of an ordinance. In French-speaking countries, the name of a ratification of a finance law generally includes the words 'ratifiant la loi de finances' or 'portant ratification de la loi de finances'.",
    "_Gen_LF_LFR": "A rectifying finance law is a legislative text (law or ordinance) adopted to replace an initial finance law. Its purpose is to revise the provisional budget during a year. The rectifying finance law may contain tax provisions that can modify the general tax code or any other tax law. In French-speaking countries, the name of a rectifying finance law generally includes the words 'portant loi de finances rectificative'.",
    "_Gen_LF_LFRmodif": "A modification of a rectifying finance law is a legislative text (law or ordinance) aimed at amending an already adopted rectifying finance law, without constituting a new rectifying finance law. In French-speaking countries, the name of a modification of a rectifying finance law generally includes the words 'portant modification de la loi de finances rectificative'.",
    "_Gen_LF_LFRratif": "The ratification of a rectifying finance law is a law passed to ratify a rectifying finance law adopted in the form of an ordinance. In French-speaking countries, the name of a ratification of a rectifying finance law generally includes the words 'ratifiant la loi de finances rectificative' or 'portant ratification de la loi de finances rectificative'.",
    "_Gen_LF_LFappli": "The application of a finance law is a regulatory text (decree or order) aimed at specifying the application methods of a finance law. In French-speaking countries, the name of the application of a finance law generally includes the words 'précisant les modalités d'application des dispositions fiscales de la loi de finances'.",
    "_Gen_LF_LFappliModif": "A modification of an application of a finance law is a regulatory text (decree or order) aimed at amending the application methods of a finance law already established in another regulatory text. In French-speaking countries, the name of a modification of the application of a finance law generally includes the words 'modifiant les modalités d'application des dispositions fiscales de la loi de finances'."
}

In [ ]:
definitions

{'_Gen_LF_LOLF': "An organic law relating to finance laws (LOLF) is a legislative text that complements and clarifies the constitutional rules that govern the development, execution, and control of finance laws. It is thus a special law that explains how finance laws must be drafted and implemented. In French-speaking countries, the name of an organic law relating to finance laws generally includes the words 'portant loi organique relative aux lois de finances' or simply 'relative aux lois de finances'.",
 '_Gen_LF_LOLFmodif': "A modification of an organic law relating to finance laws is a legislative text that amends the constitutional rules governing the development, execution, and control of finance laws. It is thus a law that modifies the organic law relating to finance laws. In French-speaking countries, the name of a modification of an organic law relating to finance laws generally includes the words 'portant modification de la loi organique relative aux lois de finances' or 'mod

In [ ]:
doc_name = "img_Loi n°2017-073 (26.12.2017) Loi de finances 2018 (Ministère des Finances)"

vector_query_engine = create_vector_engine(doc_name)

response = vector_query_engine.query(
    f"""
    Consider the contents of the document with this title '{doc_name}'. Then, these definitions: {str(definitions)} 
    In which of these categories : {', '.join(list(definitions.keys()))}, is it part of ? Answer with a valid category, keep the category only. Do not write an introduction, summary, rewrite or anything after the category.
    """
    # Answer with a valid category, keep the category only.
    # f"""
    # Considérez le contenu du document portant ce nom '{doc_name}'. Ensuite, ces définitions : {str(definitions)}
    # Dans laquelle de ces catégories : {', '.join(list(definitions.keys()))}, cela en fait-il partie ? Répondez avec une catégorie valide, gardez uniquement la catégorie.
    # """
)

# print(response)
print(doc_name, len(response.source_nodes), response.response)

img_Loi n°2017-073 (26.12.2017) Loi de finances 2018 (Ministère des Finances) 20 _Gen_LF_LF


In [ ]:
response.response

'_Gen_LF_LOLF'

# View the document contents

In [ ]:
print(f"Title: {'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé'}\nContent:\n{df[df['title']=='Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé'].iloc[0,1]}")

Title: Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé
Content:
PRESIDENCE DE LA REPUBLIQUE DU MALI
Un Peuple - Un But - Une Foi
-------------------

LOI N° 06 - 067 / DU 29 DEC. 2006
PORTANT CODE GENERAL DES IMPOTS

L'Assemblée Nationale a délibéré et adopté en sa séance du 1er décembre
2006 ;

Le Président de la République promulgue la Loi dont la teneur suit :

Titre 1 : Impôts directs

Chapitre 1 : Les impôts sur le revenu

Section I : L'impôt sur les traitements et salaires

Sous-section I : Revenus soumis à l'impôt

Article 1
Il est institué au profit du budget de l'État un Impôt sur les Traitements et
Salaires applicable à toutes les sommes payées dans l'année aux salariés par les
employeurs publics et privés, directement ou par l'entremise d'un tiers, en
contrepartie ou à l'occasion du travail, notamment à titre de traitements,
indemnités, émoluments, commissions, participations, primes, gratifications,
gages, pourboires et autres rétributions, quell

# Pipeline

In [ ]:
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)

In [ ]:
definitions = df_fisc_standards.set_index(['TexteFiscStandardCode'])['TexteFiscStandardComplet'].to_dict()
definitions.keys()

dict_keys(['_Gen_CGI', '_Gen_IT', '_Gen_VAT', '_Gen_NHI', '_Gen_LF', '_Invest_CI', '_Min_CM', '_Pétrol_CP', '_Gaz_CG', '_Forest_CF'])

In [ ]:
doc_name = 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé'

In [ ]:
# setting up the llm
llm = Ollama(
    model="llama3.1", 
    temperature=0, 
    request_timeout=180.0,
    context_window=chunk_size) 

Settings.llm = llm

In [ ]:
from llama_index.core.postprocessor import LLMRerank

from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.response_synthesizers import TreeSummarize

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.query_engine import RetrieverQueryEngine

vector_index = index

from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

def create_vector_engine(doc_name):
    filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key="title", value=doc_name
            ),
        ]
    )

    reranker = LLMRerank(
                llm=llm,
                choice_batch_size=5,
                top_n=3,
            )

    colbert_reranker = ColbertRerank(
        top_n=5,
        model="colbert-ir/colbertv2.0",
        tokenizer="colbert-ir/colbertv2.0",
        keep_retrieval_score=True,
    )

    vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=10, filters=filters) #, alpha=0.5) #, embed_model=embed_model)
    response_synthesizer = get_response_synthesizer(llm=llm, response_mode=ResponseMode.COMPACT)

    vector_query_engine = RetrieverQueryEngine(
        retriever=vector_retriever,
        response_synthesizer=response_synthesizer,
        # node_postprocessors=[colbert_reranker],
        # node_postprocessors=[processor]
    )

    return vector_query_engine

In [ ]:
filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key="title", value=doc_name
            ),
        ]
    )

vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=10, filters=filters)

In [ ]:
colbert_reranker = ColbertRerank(
        top_n=5,
        model="colbert-ir/colbertv2.0",
        tokenizer="colbert-ir/colbertv2.0",
        keep_retrieval_score=True,
    )

In [ ]:
definitions2 = df_fisc_exacts[df_fisc_exacts['TexteFiscStandardCode']==str('_Gen_CGI')].set_index(['TexteFiscExactCode'])['TexteFiscExactComplet'].to_dict()

definitions2

{'_Gen_CGI_CGI': 'Code général des impôts',
 '_Gen_CGI_CGImodif': 'Modification du Code général des impôts',
 '_Gen_CGI_CGImodifModif': 'Modification de la Modification du Code général des impôts',
 '_Gen_CGI_CGIamendé': 'Code général des impôts amendé',
 '_Gen_CGI_CGIappli': 'Application du Code général des impôts',
 '_Gen_CGI_CGIappliModif': "Modification de l'Application du Code général des impôts",
 '_Gen_CGI_CGIsuppl': 'Supplément au Code général des impôts',
 '_Gen_CGI_CGIsupplAppli': 'Application du Supplément au Code général des impôts',
 '_Gen_CGI_CGIone': 'Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneModif': 'Modification du Code des impôts directs et indirects',
 '_Gen_CGI_CGIoneAmendé': 'Code des impôts directs et indirects amendé',
 '_Gen_CGI_CGIbis': "Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobiliers",
 '_Gen_CGI_CGIbisModif': "Modification du Code de l'enregistrement, du timbre et de l'impôt sur le revenu des capitaux mobil

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core import PromptTemplate

llm = Ollama(
    model="llama3.1", 
    temperature=0, 
    request_timeout=180.0,
    context_window=chunk_size,
    context_window=16000) 

Settings.llm = llm

# define modules
prompt_str =  """
    Consider the contents of the document with this title '{doc_name}'. Then, these definitions: """ + ", ".join([f"{k}={v}" for (k,v) in definitions.items()]) + """
    In which category, is it part of ? Answer with a valid category described in the definitions above, keep the category only. Do not write an introduction, summary, rewrite or anything after the category."""
prompt_tmpl = PromptTemplate(prompt_str)
retriever = index.as_retriever(similarity_top_k=10, filters=filters)
reranker = ColbertRerank(
        top_n=5,
        model="colbert-ir/colbertv2.0",
        tokenizer="colbert-ir/colbertv2.0",
        keep_retrieval_score=True,
    )
summarizer = TreeSummarize(llm=llm)

In [ ]:
# define query pipeline
p = QueryPipeline(verbose=True)
p.add_modules(
    {
        "llm": llm,
        "prompt_tmpl": prompt_tmpl,
        "retriever": retriever,
        "summarizer": summarizer,
        "reranker": reranker,
    }
)

In [ ]:
# p.add_link("prompt_tmpl", "llm")
# p.add_link("llm", "retriever")
# p.add_link("retriever", "reranker", dest_key="nodes")
# # p.add_link("llm", "reranker", dest_key="query_str")
# p.add_link("reranker", "summarizer", dest_key="nodes")
# # p.add_link("llm", "summarizer", dest_key="query_str")

p.add_link("input", "retriever")
p.add_link("input", "summarizer", dest_key="query_str")
p.add_link("retriever", "summarizer", dest_key="nodes")

# look at summarizer input keys
print(summarizer.as_query_component().input_keys)

ValueError: Module input does not exist in pipeline.

In [ ]:
response = p.run(doc_name=doc_name)

> Running module prompt_tmpl with input: 
doc_name: Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé

> Running module llm with input: 
messages: 
    Consider the contents of the document with this title 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé'. Then, these definitions: _Gen_CGI=Code général des impôts, _Ge...

> Running module retriever with input: 
input: assistant: _Gen_CGI

> Running module reranker with input: 
query_str: assistant: _Gen_CGI
nodes: [NodeWithScore(node=TextNode(id_='caad4a58-e526-4749-b7ea-f7ed11e35809', embedding=None, metadata={'title': 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé', 'law type': '...

> Running module summarizer with input: 
query_str: assistant: _Gen_CGI
nodes: [NodeWithScore(node=TextNode(id_='95507a83-7fdb-4baf-b230-872229a68b1f', embedding=None, metadata={'title': 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé', 'law t

In [ ]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='95507a83-7fdb-4baf-b230-872229a68b1f', embedding=None, metadata={'title': 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé', 'law type': 'droit fiscal', 'retrieval_score': 0.2596357885960406}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d04b67e4-5e4a-44e0-ac80-9f7ba31e79b9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé', 'law type': 'droit fiscal'}, hash='ad04bfd8094d31bca34f68ba83f73414bcf164955c187796de428c6bf237b6a0'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='077a2a45-c0cf-44a1-b55b-daaf5ae95fdb', node_type=<ObjectType.TEXT: '1'>, metadata={'title': 'Loi n°2006-067 (29.12.2006) Code général des impôts 2006 (CNP) déverrouillé', 'law type': 'droit fiscal'}, hash='9057e4d40630f1506a4db2fd587d3a8f1cebdf5a971d2114d9a349485a404b52')

In [ ]:
## create graph
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(p.dag)
net.show("rag_dag.html")

## another option using `pygraphviz`
# from networkx.drawing.nx_agraph import to_agraph
# from IPython.display import Image
# agraph = to_agraph(p.dag)
# agraph.layout(prog="dot")
# agraph.draw('rag_dag.png')
# display(Image('rag_dag.png'))

rag_dag.html
